In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
#from folium.element import IFrame
import shapely
from shapely.geometry import Point, Polygon
import unicodedata
#import pysal as ps
from IPython.display import IFrame
import jenkspy
import seaborn as sns
import matplotlib.pyplot as plt
import branca.colormap as cm
import os,sys
from scipy import stats
import numpy as np
import pysal as ps
import seaborn as sns
def my_color_function(numberofstranding,colorpalet,threshold_scale):
    i=0
    #print(colorpalet)
    for scale in threshold_scale:
        if(numberofstranding<scale):
            return colorpalet[i]
        i=i+1
        
def add_point_clusters(mapobj, gdf, popup_field_list):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    listofcollor=["#fee186","#fdaa48","#fc5a2d","#d30f20"]
    threshold_scale=ps.esda.mapclassify.Fisher_Jenks(gdf["No_Strand"], k = 20).bins.tolist()
    marker_cluster = folium.MarkerCluster()
    print(threshold_scale)
    colorpalet=sns.color_palette("YlOrRd", 20).as_hex()
    for i, row in gdf.iterrows():
        #cluster=row["risk_cluster"]
        numberofstrand=str(row["No_Strand"])
        popup_text = """Number of Stranding: {}"""
        popup_text = popup_text.format(numberofstrand)
        label = 'Number of Stranding : '+numberofstrand
        color=my_color_function(row["No_Strand"],colorpalet,threshold_scale)
        circlemarker1=folium.CircleMarker([row.geometry.y, row.geometry.x], popup=popup_text, color=color,fill_color=color, fill_opacity=3, weight=4,radius=2).add_to(mapobj)
        

    minval=gdf['No_Strand'].min()
    maxval=gdf['No_Strand'].max()

    colormap=cm.linear.YlOrRd.scale(minval, maxval)
    colormap.caption = 'Number of Stranding'
    mapobj.add_child(colormap)


    return mapobj
def to_number(s):
    try:
        #print(s)
        s1 = float(s)
        return s1
    except ValueError:
        #print(s)
        return 0

whalestraningdf=pd.read_csv('fulldatefile.csv',keep_default_na=False)
whalestraningdf=whalestraningdf[(whalestraningdf["No_Strand"]>5)] [["date","Lat","Long","No_Strand"]]

#whalestraningdf['Lat'] = whalestraningdf['Lat'].astype(float)
whalestraningdf['Lat']=whalestraningdf.apply(lambda f : to_number(f['Lat']) , axis = 1)  
#whalestraningdf['Lat'] = pd.to_float(whalestraningdf['Lat'])

whalestraningdf['Long']=whalestraningdf.apply(lambda f : to_number(f['Long']) , axis = 1)  
whalestraningdf=whalestraningdf[(whalestraningdf["Lat"]!=0.0)][["date","Lat","Long","No_Strand"]]
whalestraningdf=whalestraningdf[(whalestraningdf["Long"]!=0.0)][["date","Lat","Long","No_Strand"]]
#whalestraningdf['Long'] = pd.to_float(whalestraningdf['Long'])
longitude= whalestraningdf['Long'].median()
latitude= whalestraningdf['Lat'].median()
#print(whalestraningdf)
whalestraning_geo = gpd.GeoSeries(whalestraningdf.apply(lambda z: Point(z['Long'], z['Lat']), 1),crs={'init': 'epsg:4326'})
whalestraningdf = gpd.GeoDataFrame(whalestraningdf.drop(['Lat', 'Long'], 1), geometry=whalestraning_geo)

print(longitude)
print(latitude)
whalestraning_map = folium.Map([-40.9006,longitude], zoom_start = 5)

whalestraning_map = add_point_clusters(whalestraning_map, whalestraningdf, ['No_Strand'])


    
#    }
#).add_to(violation_map)

#folium.LayerControl().add_to(violation_map)
whalestraning_map.save('WhaleStrandingNewZealandMoreThan5.html')

whalestraning_map


